<a href="https://colab.research.google.com/github/trodman201/Data-2000-/blob/homework/LinearModelsRodman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Part 1:
The year, manufacturer, model, condition, fuel and transmission can have a direct impact on the vehicle's pricing. Meanwhile the odometer could have an impact on the car's value.  

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


car_data_raw = pd.read_csv("https://cdn.c18l.org/vehicles_lab.csv")

df = car_data_raw[['year', 'manufacturer', 'model', 'odometer', 'condition', 'fuel', 'transmission', 'price']]

Part 2: I decided to dummy code manufacturer, model, condition, fuel, transmission so that these categorical variables can be read as binary/dichotomous instead and so it would be easier to model them. One hot encoding increases the dimensionality of the data and to no ones surprise it kept crashing.


In [3]:
scaler = MinMaxScaler()
df[['year', 'odometer']] = scaler.fit_transform(df[['year', 'odometer']])

imputer = SimpleImputer(strategy='most_frequent')
categorical_cols = ['manufacturer', 'model', 'fuel', 'transmission', 'condition']
df[categorical_cols] = imputer.fit_transform(df[categorical_cols])

df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
df.head()

<ipython-input-3-a30ebf07168e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['year', 'odometer']] = scaler.fit_transform(df[['year', 'odometer']])
<ipython-input-3-a30ebf07168e>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical_cols] = imputer.fit_transform(df[categorical_cols])


,year,odometer,price,manufacturer_alfa-romeo,manufacturer_aston-martin,manufacturer_audi,manufacturer_bmw,manufacturer_buick,manufacturer_cadillac,manufacturer_chevrolet,...,fuel_gas,fuel_hybrid,fuel_other,transmission_manual,transmission_other,condition_fair,condition_good,condition_like new,condition_new,condition_salvage
0,0.926230,0.01280,15000,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0.918033,0.00687,27990,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,0,0
2,0.950820,0.00295,34590,0,0,0,0,0,0,1,...,1,0,0,0,1,0,1,0,0,0
3,0.975410,0.00430,35000,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0.950820,0.00173,29990,0,0,0,0,0,0,1,...,1,0,0,0,1,0,1,0,0,0


Part 3: I wanted to create one column of the age of the car so one could see how depreciation can impact the value. Another column I wanted to create was the mileage per year so that the vehicles condition can also be determined by the average annual mileage.

In [4]:
df['age_of_vehicle'], df['mileage_per_year'] = pd.Timestamp.now().year - df['year'], df['odometer'] / (pd.Timestamp.now().year - df['year'])

In [ ]:
price_bins = [0, 9999, 19999, 29999, 39999, np.inf]
labels = ['$0-$9,999', '$10,000-$19,999', '$20,000-$29,999', '$30,000-$39,999', '$40,000+']

df['price_bin'] = pd.cut(df['price'].astype(str).str.replace('[\$,]', '', regex=True).astype(float), bins=price_bins, labels=labels)
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['price', 'price_bin']), LabelEncoder().fit_transform(df['price_bin']), test_size=0.2, random_state=42)

print("Shape of X:", X_train.shape)
print("Shape of y:", y_train.shape)

In [ ]:
model = LogisticRegression(multi_class='multinomial', max_iter=1000)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy}")
print(f"Testing Accuracy: {test_accuracy}")


Part 4: this entire model works whenever it wants to like its sentient, no other ideas on how to be more space efficient. The good thing is that I found out colab has a kitty mode and cats walk on top of the screen  (tools > settings > misc) oh. theres also a puppy mode if you like dogs.